<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
#!pip install psycopg2
#!pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org --trusted-host pypi.python.org psycopg2
import pandas as pd
import psycopg2

In [2]:
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [3]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [41]:
query_3_1 = f'''
SELECT *    
  FROM vacancies
'''

In [55]:
vacancy_df = pd.read_sql_query(query_3_1, connection)
print('Количество вакансий -', vacancy_df['id'].nunique())

Количество вакансий - 49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [47]:
query_3_2 = f'''
SELECT *    
  FROM employers
'''

In [48]:
employers_df = pd.read_sql_query(query_3_2, connection)
print('Количество работодателей -', employers_df['id'].nunique())

Количество работодателей - 23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [49]:
query_3_3 = f'''
SELECT *    
  FROM areas
'''

In [50]:
areas_df = pd.read_sql_query(query_3_3, connection)
print('Количество регионов -', areas_df['id'].nunique())

Количество регионов - 1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [53]:
query_3_4 = f'''
SELECT *    
  FROM industries
'''

In [54]:
industry_df = pd.read_sql_query(query_3_4, connection)
print('Количество сфер деятельности -', industry_df['id'].nunique())

Количество сфер деятельности - 294


***

Из представленной выше статистики можно сделать следующие выводы:
* База данных содержит обширный набор данных, который включает в себя:
    * 49197 уникальных вакансий от 23501 уникальных работодателей.
    * Вышеперечисленные вакансии представленны в 294-х различных сферах деятельности и расположены в 1362-х регионах.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [64]:
query_3_5 = f'''
SELECT 
COUNT(v.id) cnt,
a.name  area   
  FROM vacancies v
  JOIN areas a ON v.area_id = a.id
  GROUP BY a.id 
ORDER BY  cnt DESC
'''

In [69]:
query_3_5_df = pd.read_sql_query(query_3_5, connection)
print('Пятёрку лидеров по количеству вакансий:\n',  query_3_5_df.head(5))

Пятёрку лидеров по количеству вакансий:
     cnt             area
0  5333           Москва
1  2851  Санкт-Петербург
2  2112            Минск
3  2006      Новосибирск
4  1892           Алматы


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [80]:
query_3_6 = f'''
SELECT 
id,
salary_from,
salary_to  
  FROM vacancies v
   WHERE salary_from IS NOT NULL OR salary_to IS NOT NULL
'''

In [81]:
query_3_6_df = pd.read_sql_query(query_3_6, connection)
print('Количество вакансий у которых заполнено хотя бы одно из двух полей с зарплатой -',  query_3_6_df.shape[0])

Количество вакансий у которых заполнено хотя бы одно из двух полей с зарплатой - 24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [84]:
query_3_7 = f'''
SELECT 
AVG(salary_from) salary_bottom,
AVG(salary_to) salary_top
  FROM vacancies v
'''

In [91]:
query_3_7_df = pd.read_sql_query(query_3_7, connection)
print('Среднее значение для нижней границы зарплаты -',  round(query_3_7_df['salary_bottom'][0], 0))
print('Среднее значение для верхней границы зарплаты -',  round(query_3_7_df['salary_top'][0], 0))

Среднее значение для нижней границы зарплаты - 71065.0
Среднее значение для верхней границы зарплаты - 110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [92]:
query_3_8 = f'''
SELECT 
COUNT(*) num_vacancies,
employment, 
schedule
  FROM vacancies v
  GROUP BY employment, schedule
  ORDER BY num_vacancies DESC
'''

In [128]:
query_3_8_df = pd.read_sql_query(query_3_8, connection)
query_3_8_df[query_3_8_df['num_vacancies']==query_3_8_df['num_vacancies'].nlargest(2)[1]][['schedule','employment']]

,schedule,employment
1,Удаленная работа,Полная занятость


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [117]:
query_3_9 = f'''
SELECT 
COUNT(*) num_vacancies,
experience
  FROM vacancies v
  GROUP BY experience
  ORDER BY num_vacancies ASC
'''

In [118]:
query_3_9_df = pd.read_sql_query(query_3_9, connection)
print('Требуемый опыт работы в порядке возрастания: \n',  query_3_9_df['experience'].iloc[:4])


Требуемый опыт работы в порядке возрастания: 
 0           Более 6 лет
1             Нет опыта
2         От 3 до 6 лет
3    От 1 года до 3 лет
Name: experience, dtype: object


***

Из представленной выше статистики можно сделать следующие выводы:
* Количество вакансий в крупных населенных пунктах имеет положительную ассоциацию с количеством населения в городах (вполне логично):
* Около половины всех вакансий указывыют хотя бы один из признаков зарплаты (верхний или нижний предел).
* Касательно предела зарплат, то средняя величина верхней границы заплаты (110537) в полтора раза больше нижней границы заплаты (71065).
* Наиболее популярным сочетанием типа рабочего графика и типа трудоустройства является - "Полный день" и "Полная занятость". Вторым по популятности является "Удаленная работа" и "Полная занятость". Наименее популярной является "Сменный график" и "Проектная работа".
* Наиболее популярным опытом работы исходя из количества вакансий является "От 1 года до 3 лет". Тогда как наименее популярным является "Более 6 лет".

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [11]:
query_3_10 = f'''
SELECT 
COUNT(v.id) num_vacancies,
e.name
  FROM employers e
  JOIN vacancies v ON e.id=v.employer_id
  GROUP BY e.id
  ORDER BY num_vacancies DESC
  LIMIT 5
'''

In [12]:
query_3_10_df = pd.read_sql_query(query_3_10, connection)
print('Работодатель, который находится на первом месте по количеству вакансий -', query_3_10_df['name'][0] )
print('Работодатель, который находится на пятом месте по количеству вакансий -', query_3_10_df['name'][4])

Работодатель, который находится на первом месте по количеству вакансий - Яндекс
Работодатель, который находится на пятом месте по количеству вакансий - Газпром нефть


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [28]:
query_3_11 = f'''
SELECT 
a.name region,
COUNT(e.id) num_employers,
COUNT(v.id) num_vacancies
  FROM areas a
  LEFT JOIN employers e ON e.area=a.id
  LEFT JOIN vacancies v ON a.id=v.area_id
  GROUP BY a.id
'''

In [58]:
query_3_11_df = pd.read_sql_query(query_3_11, connection)
no_vacan_index= query_3_11_df[query_3_11_df['num_vacancies']==0]['num_employers'].nlargest(1).index
print('Среди регионов, в которых нет вакансий, наибольшее количество работодателей находится в регионе под названием -', query_3_11_df['region'].iloc[no_vacan_index].iloc[0])

Среди регионов, в которых нет вакансий, наибольшее количество работодателей находится в регионе под названием - Россия


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [71]:
query_3_12 = f'''
SELECT
COUNT(DISTINCT v.area_id) num_regions
  FROM vacancies v
  WHERE employer_id IS NOT NULL and area_id IS NOT NULL
  GROUP BY employer_id
  ORDER BY num_regions DESC
'''

In [72]:
query_3_12_df = pd.read_sql_query(query_3_12, connection)
print('Махимальное количество регионов, к которых работодатель публикует свои вакансии -', query_3_12_df['num_regions'].max())

#Также можно использовать код ниже (т.к. мы уже отсортировали по количеству регионов в запросе)
#print('Махимальное количество регионов, к которых работодатель публикует свои вакансии -', query_3_12_df['num_regions'].iloc[0])

Махимальное количество регионов, к которых работодатель публикует свои вакансии - 181


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [74]:
query_3_13 = f''' 
SELECT 
COUNT(DISTINCT e.id) employer_num
  FROM employers e
  LEFT JOIN employers_industries e_i ON e_i.employer_id=e.id
  WHERE industry_id IS NULL
  GROUP BY employer_id
  ORDER BY employer_num DESC
'''

In [75]:
query_3_13_df = pd.read_sql_query(query_3_13, connection)
print('Количества работодателей, у которых не указана сфера деятельности -', query_3_13_df['employer_num'].iloc[0])

Количества работодателей, у которых не указана сфера деятельности - 8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [77]:
query_3_14 = f''' 
SELECT 
e.name company_name
  FROM employers e
  LEFT JOIN employers_industries e_i ON e_i.employer_id=e.id
  WHERE industry_id IS NOT NULL
  GROUP BY e.id
  HAVING COUNT(e.id)=4
  ORDER BY company_name ASC
  OFFSET 2 LIMIT 1
''' 

In [78]:
query_3_14 = pd.read_sql_query(query_3_14, connection)
print('Название компании находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности -', query_3_14['company_name'].iloc[0])

Название компании находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности - 2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [80]:
query_3_15 = f''' 
SELECT 
COUNT(*) employers_num
  FROM employers e
  LEFT JOIN employers_industries e_i ON e_i.employer_id=e.id
  LEFT JOIN industries i ON i.id=e_i.industry_id
  WHERE i.name = 'Разработка программного обеспечения'
'''  

In [81]:
query_3_15 = pd.read_sql_query(query_3_15, connection)
print('Количество работодателей, у которых в качестве сферы деятельности указана «Разработка программного обеспечения» -', query_3_15['employers_num'].iloc[0])

Количество работодателей, у которых в качестве сферы деятельности указана «Разработка программного обеспечения» - 3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [248]:
# код для получения списка городов-милионников
import requests # Импортируем библиотеку requests
from bs4 import BeautifulSoup # Импортируем библиотеку BeautifulSoup
import re

url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8' # Определяем адрес страницы
response = requests.get(url)
page = BeautifulSoup(response.text, 'html.parser')
string_of_cities = page.find('table', class_='standard').text
clean_strings = re.findall('[А-Яа-я-\s]+', string_of_cities)
clean_strings = [ele for ele in clean_strings if ele != ' '] # Удаляем элементы, которые равны пустым значениям
#Создаем кортеж из городов миллионников, чтобы использовать его в SQL запросе
city_tuple = tuple(clean_strings[11:]) #Отбрасываем первый 11 значений в clean_strings, т.к. это текст не относится к городам

In [249]:
query_3_16 = f''' 
SELECT 
a.name,
COUNT(v.id) cnt
  FROM vacancies v 
  JOIN employers e ON e.id=v.employer_id
  JOIN areas a ON a.id=v.area_id
  WHERE e.name = 'Яндекс' AND a.name IN {city_tuple}
GROUP BY a.name  

UNION ALL
SELECT
'Total', COUNT(v.id)
  FROM vacancies v 
  JOIN employers e ON e.id=v.employer_id
  JOIN areas a ON a.id=v.area_id
  WHERE e.name = 'Яндекс' AND a.name IN {city_tuple}
GROUP BY e.name
ORDER BY cnt 
'''

In [250]:
query_3_16 = pd.read_sql_query(query_3_16, connection)
#print("Фильная таблица: \n", query_3_16)
print("Финальная таблица: \n", query_3_16)
print('\n')
print("Количество строк в таблице -", query_3_16.shape[0])
print("Значение строки `Total' -", query_3_16[query_3_16['name'] == 'Total']['cnt'].iloc[0])


Фильная таблица: 
                name  cnt
0              Омск   21
1         Челябинск   22
2        Красноярск   23
3         Волгоград   24
4             Пермь   25
5            Казань   25
6    Ростов-на-Дону   25
7               Уфа   26
8            Самара   26
9         Краснодар   30
10          Воронеж   32
11      Новосибирск   35
12  Нижний Новгород   36
13     Екатеринбург   39
14  Санкт-Петербург   42
15           Москва   54
16            Total  485


Количество строк в таблице -  17
Значение строки `Total' -  485


***

Из представленной выше статистики можно сделать следующие выводы:
* Топ-5 работодателей (по количеству предлагаемых работ) - Яндекс, Ростелеком, Тинькофф, СБЕР, Газпром нефть. На них суммарно приходится 3627 вакансии или 7.4%. 
* Топ-5 регионов по количеству вакансий – Москва, Санкт-Петербург, Минск, Алматы, Новосибирск (код: Приложение-1). 
* Наибольшее количество вакансий, которые публикуют регионы, равно – 181.
* Общее число работодателей, у которых не указана сфера деятельности равна – 8419.
* Топ-5 компаний с наибольшим числом сфер деятельности (5) - Группа компаний МегаМейд, КЗ ПромКотлоСнаб, Транзит, Фирма Гера,  Этажи, федеральная компания (код: Приложение-2).
* Разработкой программного обеспечения занимаются 3553 работодателей. 
* Компания Яндекс предлагает вакансии во всех городах-миллионников (г-м), с наибольшим количеством вакансий в Москве (54) и общим количеством вакансий в г-м равным 485.


# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [ ]:
# текст запроса

In [ ]:
# результат запроса

2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [ ]:
# текст запроса

In [ ]:
# результат запроса

3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [ ]:
# текст запроса

In [ ]:
# результат запроса

4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [ ]:
# текст запроса

In [ ]:
# результат запроса

5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [ ]:
# текст запроса

In [ ]:
# результат запроса

6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [ ]:
# текст запроса

In [ ]:
# результат запроса

***

In [ ]:
# выводы по предметному анализу

# Общий вывод по проекту

In [ ]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования

### Далее представлен дополнительный код, который использовался для доп. выводов:

* Приложение-1

In [ ]:
query_3_11_df['region'].iloc[query_3_11_df['num_vacancies'].nlargest(5).index]

* Приложение-2

In [ ]:
query_3_14_Extra = f''' 
SELECT 
e.name company_name,
COUNT(e_i.industry_id) num_industries
  FROM employers e
  LEFT JOIN employers_industries e_i ON e_i.employer_id=e.id
  WHERE industry_id IS NOT NULL
  GROUP BY e.id
  ORDER BY num_industries DESC
''' 

query_3_14_extra = pd.read_sql_query(query_3_14_Extra, connection)
query_3_14_extra.head(5)